In [ ]:
# Data Load
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

loader = TextLoader('llm.txt')

text_docs = loader.load()

text_docs = text_splitter.split_documents(text_docs)

text_docs

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
import bs4


loader=WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                         class_=("post-title","post-content","post-header")

                     )))

docs = loader.load()


In [ ]:
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
embeddings = OllamaEmbeddings(model="llama3.1")

db = Chroma.from_documents(text_docs, embeddings)

In [ ]:
query = "what is dog?"
retreived_results = db.similarity_search(query)
print(retreived_results)

In [ ]:
from langchain_community.vectorstores import LanceDB
import lancedb

lanceDB = lancedb.connect("tmp/lancedb")

table = lanceDB.create_table("my_table",data=[
    {
        "vector":embeddings.embed_query("text_docs"),
        "text":"test text",
        "id":"1"
    }
], mode="overwrite")

lanceDB = LanceDB.from_documents(text_docs, embeddings)



In [33]:

query = "llm?"
retreived_results = lanceDB.similarity_search(query)
print(retreived_results[0].page_content)

A large language model (LLM) is a type of artificial intelligence (AI) program that can recognize and generate text, among other tasks. LLMs are trained on huge sets of data â€” hence the name "large." LLMs are built on machine learning: specifically, a type of neural network called a transformer model.

In simpler terms, an LLM is a computer program that has been fed enough examples to be able to recognize and interpret human language or other types of complex data. Many LLMs are trained on data that has been gathered from the Internet â€” thousands or millions of gigabytes' worth of text. But the quality of the samples impacts how well LLMs will learn natural language, so an LLM's programmers may use a more curated data set.
No words from the query are present in the result string.
